In [1]:
import re
import requests
import bs4
import json
import pandas as pd
import time

from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options

from threading import Thread

In [2]:
#we will use 2 hreads, one to scrape the house section and one the apprtment sections
#this will speed up the process, and those 2 lists will make sure there are no data races
all_links_house = []
all_links_appartment = []

In [3]:
def get_html(link):
    """
    input : a link
    return the html code (with javascript) of the page 
    """
    #!!!!!manage cookies!!!!!!!
    path = r'C:\Users\Sacha\Documents\BeCode\geckodriver-v0.32.0-win32\geckodriver.exe' #to be change for diff computers
    c = Options()
    c.add_argument("--headless")  #  faire fonctionner headless!!!!!!!!!!!!!!!!!!!!
    
    driver = webdriver.Firefox(executable_path = path, options=c )
    driver.get(link)
    
    html = driver.page_source;
    
    driver.close()
    
    return html

In [4]:
def get_link_from_html(html):
    """
    input: an html content
    scrape the html content to get tthe links of the house/appartment cards of that pag
    return a list with those links
    """
    temp = []
    soup = bs4.BeautifulSoup(html, "html.parser")
    for card in soup.find_all("a", class_="card__title-link"):
        temp.append(card['href'])
    return temp
    

In [5]:
def get_link_house(base_house_link, end_link, start, end):
    """
    input: base_house_link/ end_link = part of a link
           start/end = integer representing the start and the end of a loop
    returns all the links for houses between pages start and end
    """
    global all_links_house
    for i in range (start,end):
        print("house, page",i)
        link = base_house_link + str(i) + end_link
        html = get_html(link)
        all_links_house.extend(get_link_from_html(html))

In [6]:
def get_link_appartment(base_appartment_link, end_link, start, end):
    """
    input: base_house_link/ end_link = part of a link
           start/end = integer representing the start and the end of a loop
    returns all the links forappartment between pages start and end
    """
    global all_links_appartment
    for i in range (start,end):
        print("appart, page",i)
        link = base_appartment_link + str(i) + end_link
        html = get_html(link)
        all_links_appartment.extend(get_link_from_html(html))

In [7]:
def get_all_links():
    """
    this function scrape immoweb to get all the links of appartments and houses to sale in belgium, and it saves it
    """
    start = time.time()
    base_house_link = "https://www.immoweb.be/en/search/house/for-sale?countries=BE&page="
    base_appartment_link = "https://www.immoweb.be/en/search/apartment/for-sale?countries=BE&page="
    num = "0"
    end_link = "&orderBy=relevance"
    
    all_links = []
    
    """
    #at first, we will take every link in the house section
    for i in range (1,334):
        print("house, page",i)
        link = base_house_link + str(i) + end_link
        html = get_html(link)
        all_links.extend(get_link_from_html(html))
        
    #then we will take all the link in the appartment section
    for i in range (1,334):
        print("appartment, page",i)
        link = base_appartment_link + str(i) + end_link
        html = get_html(link)
        all_links.extend(get_link_from_html(html))
    """
    
    ##################################### with concurrency #############################################
    threads = list()
    
    #at first, we will take every link in the house section
    thread = Thread(target=get_link_house, args=(base_house_link, end_link, 1, 334, )) 
    threads.append(thread) 
    #then, we will take every link in the appartment section
    thread = Thread(target=get_link_appartment, args=(base_appartment_link, end_link, 1, 334, )) 
    threads.append(thread) 
    
    for thread in threads:
        thread.start()
    for thread in threads:  
        thread.join()
        
    all_links.extend(all_links_house)
    all_links.extend(all_links_appartment)
    ################################################################################################
    
    df = pd.DataFrame(all_links) #all_links is a list of 1 list
    unique_links_df = df.duplicated(keep='first')
    df = df[~unique_links_df]       #delete the duplicates
    
    print("num duplicates",unique_links_df.sum())
    #print(df.iloc[0,0])
    print("size",df.size)
    
    df.to_csv('links.csv')
    end = time.time()
    print("done", end-start)

In [1]:
get_all_links()